## Install required packages

In [5]:
!pip install --upgrade pycaret scikit-plot azureml-train-automl azureml-mlflow azureml-core azureml-dataprep azureml-widgets

packages (from azureml-automl-runtime~=1.18.0->azureml-train-automl) (1.6.0)
  Attempting uninstall: azureml-dataprep-rslex
    Found existing installation: azureml-dataprep-rslex 1.2.3
    Uninstalling azureml-dataprep-rslex-1.2.3:
      Successfully uninstalled azureml-dataprep-rslex-1.2.3
  Attempting uninstall: azureml-dataprep-native
    Found existing installation: azureml-dataprep-native 24.0.0
    Uninstalling azureml-dataprep-native-24.0.0:
      Successfully uninstalled azureml-dataprep-native-24.0.0
  Attempting uninstall: azureml-dataprep
    Found existing installation: azureml-dataprep 2.4.4
    Uninstalling azureml-dataprep-2.4.4:
      Successfully uninstalled azureml-dataprep-2.4.4
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default

## PyCaret local

In [6]:
import mlflow
from azureml.core import Workspace

ws = Workspace.from_config()
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

/Users/cody/miniconda3/envs/automl/lib/python3.7/site-packages/urllib3/util/retry.py:439: DeprecationWarning: Using 'method_whitelist' with Retry is deprecated and will be removed in v2.0. Use 'allowed_methods' instead
  DeprecationWarning,


In [7]:
import pandas as pd 

df = pd.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/iris.csv")
df.head()

/Users/cody/miniconda3/envs/automl/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [8]:
df.shape

/Users/cody/miniconda3/envs/automl/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(150, 5)

In [9]:
data = df.sample(frac=0.95, random_state=42)
data_unseen = df.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print("Data for modeling: " + str(data.shape))
print("Unseen data for predictions: " + str(data_unseen.shape))

Data for modeling: (142, 5)
Unseen data for predictions: (8, 5)


In [21]:
from pycaret.classification import *

exp = setup(
    data=data,
    target="species",
    log_experiment=True,
    experiment_name="pycaret-untitled",
    log_plots=True,
    log_profile=True,
    silent=False,  # set to False for interactively setting data types
)

,Description,Value
0,session_id,2943
1,Target,species
2,Target Type,Multiclass
3,Label Encoded,"Iris-setosa: 0, Iris-versicolor: 1, Iris-virginica: 2"
4,Original Data,"(142, 5)"
5,Missing Values,False
6,Numeric Features,4
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


UnsupportedModelRegistryStoreURIException:  Model registry functionality is unavailable; got unsupported URI 'azureml://eastus.experiments.azureml.net/mlflow/v1.0/subscriptions/6560575d-fa06-4e7d-95fb-f962e74efd7a/resourceGroups/azureml-examples/providers/Microsoft.MachineLearningServices/workspaces/default?' for model registry data storage. Supported URI schemes are: ['', 'file', 'databricks', 'http', 'https', 'postgresql', 'mysql', 'sqlite', 'mssql']. See https://www.mlflow.org/docs/latest/tracking.html#storage for how to run an MLflow server against one of the supported backend storage locations.

In [22]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,pycaret.internal.tunable.TunableMLPClassifier,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [23]:
%%time
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.9900,1.0000,0.9889,0.9920,0.9896,0.9846,0.9860,0.0080
qda,Quadratic Discriminant Analysis,0.9800,1.0000,0.9806,0.9845,0.9796,0.9697,0.9723,0.0080
lr,Logistic Regression,0.9700,1.0000,0.9694,0.9792,0.9670,0.9538,0.9601,0.2010
knn,K Neighbors Classifier,0.9700,1.0000,0.9694,0.9792,0.9670,0.9538,0.9601,0.0480
nb,Naive Bayes,0.9700,1.0000,0.9694,0.9765,0.9691,0.9543,0.9583,0.0070
gbc,Gradient Boosting Classifier,0.9700,0.9690,0.9694,0.9792,0.9670,0.9538,0.9601,0.0550
et,Extra Trees Classifier,0.9700,1.0000,0.9694,0.9792,0.9670,0.9538,0.9601,0.1650
xgboost,Extreme Gradient Boosting,0.9700,0.9985,0.9694,0.9792,0.9670,0.9538,0.9601,0.0240
dt,Decision Tree Classifier,0.9600,0.9679,0.9583,0.9712,0.9566,0.9384,0.9460,0.0080
rf,Random Forest Classifier,0.9600,0.9986,0.9583,0.9712,0.9566,0.9384,0.9460,0.1630


CPU times: user 7.45 s, sys: 202 ms, total: 7.65 s
Wall time: 18.5 s


In [24]:
print(best_model)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)


In [25]:
evaluate_model(best_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [26]:
unseen_predictions = predict_model(best_model, data=data_unseen)
unseen_predictions.head()

,sepal_length,sepal_width,petal_length,petal_width,species,Label,Score
0,5.8,4.0,1.2,0.2,Iris-setosa,Iris-setosa,1.0
1,5.4,3.4,1.7,0.2,Iris-setosa,Iris-setosa,1.0
2,6.1,2.8,4.0,1.3,Iris-versicolor,Iris-versicolor,1.0
3,5.8,2.6,4.0,1.2,Iris-versicolor,Iris-versicolor,1.0
4,7.1,3.0,5.9,2.1,Iris-virginica,Iris-virginica,1.0


In [27]:
from pycaret.utils import check_metric

check_metric(
    unseen_predictions.species,
    unseen_predictions.Label.astype(str),
    "Accuracy",
)

1.0

## Azure AutoML local

In [28]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

Workspace.create(name='default', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='azureml-examples')

In [30]:
from azureml.core import Dataset 

tab = Dataset.Tabular.from_delimited_files(["https://azuremlexamples.blob.core.windows.net/datasets/iris.csv"])
tab

{
  "source": [
    "https://azuremlexamples.blob.core.windows.net/datasets/iris.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

## Downgrade various pip packages

In [32]:
!pip install azureml-automl-runtime

me[fuse,pandas]~=1.18.0 in /Users/cody/miniconda3/envs/automl/lib/python3.7/site-packages (from azureml-automl-runtime) (1.18.0)
     |████████████████████████████████| 678 kB 18.8 MB/s 
  Using cached joblib-0.14.1-py2.py3-none-any.whl (294 kB)
     |████████████████████████████████| 10.2 MB 23.3 MB/s 
     |████████████████████████████████| 1.4 MB 46.1 MB/s 
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Using cached azureml_dataprep-2.4.4-py3-none-any.whl (28.2 MB)
  Using cached azureml_dataprep_rslex-1.2.3-cp37-cp37m-macosx_10_9_x86_64.whl (7.4 MB)
  Using cached azureml_dataprep_native-24.0.0-cp37-cp37m-macosx_10_9_x86_64.whl (145 kB)
  Created wheel for smart-open: filename=smart_open-1.9.0-py3-none-any.whl size=73087 sha256=1c412268fba14851fc96386e8ee2dc5e0f4484862a83b421db6a4c784a05947b
  Stored in directory: /Users/cody/Library/Caches/pip/wheels/1f/f6/d9/49cfa288fb14de91f127fb6f7a5a29f389c9588a41140107d9
Successfully built smart-open
  Attempting uninstall: sm

In [34]:
from azureml.train.automl import AutoMLConfig

config = AutoMLConfig(
    task="classification",
    compute_target="local",
    training_data=tab,
    label_column_name="species"
)

from azureml.core import Experiment

run = Experiment(ws, "automl-unittled").submit(config, show_output=True)
run

ValidationException: ValidationException:
	Message: Install the required versions of packages using the requirements file. Requirements file location: /Users/cody/miniconda3/envs/automl/lib/python3.7/site-packages/azureml/automl/core/validated_darwin_requirements.txt. Alternatively, use remote target to avoid dependency management. 
Package name/Required version/Installed version
azureml-automl-runtime/smart-open<=1.9.0/smart-open 3.0.0
azureml-automl-runtime/numpy<1.19.0,>=1.16.0/numpy 1.19.4
azureml-automl-runtime/statsmodels<=0.10.2,>=0.9.0/statsmodels 0.12.1
azureml-automl-runtime/pandas<1.0.0,>=0.21.0/pandas 1.1.4
azureml-automl-runtime/lightgbm<=2.3.0,>=2.0.11/lightgbm 3.1.0
azureml-automl-runtime/scipy<=1.5.2,>=1.0.0/scipy 1.5.4
azureml-automl-runtime/scikit-learn<0.23.0,>=0.19.0/scikit-learn 0.23.2
azureml-automl-runtime/joblib==0.14.1/joblib 0.17.0
azureml-defaults/gunicorn==19.9.0/gunicorn 20.0.4
azureml-defaults/flask==1.0.3/Flask 1.1.2
azureml-train-automl-runtime/smart-ope